# So, we have data. Now what?

Now we slice it and then we cook it :D

At this point, the input we are getting from all these __streamers__ or __readers__ are documents or sentences, usually as Python strings. That is a start but it still does not count as NLP :)

The main problem is that string are too large to be useful: for whole documents, it is very unlikely our system will ever come across the same string. Therefore, even if we trained it to perform a perfect action for that document, it would never get a chance to perform that action. That means zero generalization and no automation. In order to really automate something, we need to ensure it is repeatable.

At the core of repeatability lies resolution: the higher our system's resolution, and the more parts of a unit it is sensitive to, the greater the likelihood that it will find a pattern shared by its training data that can lead it to provide the correct answer.

The process of reducing strings into smaller subunits is called __feature extraction__ and it is the first step towards turning something a person wrote into something a computer can understand.


# Basics of __feature extraction__
Some of the most trivial preprocessing steps we can perform are sentence splitting and word tokenization. Again, NLTK can help us with that. Let's see some examples from the Reuters corpus:

In [ ]:
import nltk

from nltk.corpus import reuters

i = list(reuters.fileids())[10]
print reuters.raw(i)

In [ ]:
print reuters.sents(i.encode('utf-8'))

In [ ]:
print reuters.words(i.encode('utf-8'))

Instances of the NLTK's CorpusReader class will come with built-in methods for accessing this information, which will therefore be available for the corpus supported in NLTK. What if we are given a new dataset, not in NLTK?

We will still use NLTK :) (told you, it helps ;) More specifically, methods such as __wordpunct_tokenize__ and __sent_tokenize__. Let's pretend the Reuters text is our new text:

In [ ]:
from nltk import (
    sent_tokenize as splitter,
    wordpunct_tokenize as tokenizer
)

text = """SUBROTO SAYS INDONESIA SUPPORTS TIN PACT EXTENSION
  Mines and Energy Minister Subroto
  confirmed Indonesian support for an extension of the sixth
  International Tin Agreement (ITA), but said a new pact was not
  necessary.
      Asked by Reuters to clarify his statement on Monday in
  which he said the pact should be allowed to lapse, Subroto said
  Indonesia was ready to back extension of the ITA.
      "We can support extension of the sixth agreement," he said.
  "But a seventh accord we believe to be unnecessary."
      The sixth ITA will expire at the end of June unless a
  two-thirds majority of members vote for an extension."""

print splitter(text)[:2]

In [ ]:
print [tokenizer(sent) for sent in splitter(text)[:2]]

As you can see, we can easily use NLTK's methods to get the sentences in a text, or the words in that text. Some systems do not need sentences as input and are designed to work with the individual words in each document (text classification, sentiment analysis, or term extraction can all work either at the sentence or the document level with little difference overall). However, sentences will be useful when building statistical models like n-gram models (we will see that in more detail in a bit).

## Tokens
The term _words_ we used in the previous paragraph is technically incorrect. At this level, we are still working with __tokens__, which include all words in a text plus any punctuation signs, plus any numerical expressions, plus virtually anything else. Strictly speaking, the number "103" is not a word but it can appear in a text. Therefore, it is a __token__. A __token__ refers to anything that appears within __word boundaries__. Word boundaries are not always well-defined and even today there are inconsistencies in the way some tokens are handled by different systems: some consider _didn't_ two separate words, and tokenize it as _did not_, whereas others take it as a single word. For our purposes, _did not_ is preferable to _didn't_ but, generally speaking, we would like our system to be robust to this type of thing. That is, we don't want to have to teach it explicitly that _did not_ and _didn't_ are two variants of the same linguistic unit. We want the system to infer that from their behavior as observed in the data. That will be when we will be able to claim we are doing something close to machine learning.

## Punctuation
Although some advanced applications can handle non-word tokens such as punctuation (for instance, to skip parenthetical elements inside a sentence, or appositions), most applications do not do much with it once the sentences have been split. Generally speaking, we can ignore punctuation in most tasks and we will still get the desired results, assuming we can train a good sequence model (we will see what this means later).

## Numerical expressions
The case of numerical expressions is more complicated because most systems tend to remove any patterns involving digits, which is fine in many cases. Sometimes, however, and depending on the task you are working on, numerical information will be crucial for the system: in many applications in which time information is important (for example, a website for booking flights needs to detect and process the dates of a trip), numerical information cannot be removed. Instead, a specific parser is usually required to handle each particular type of information: temporal expressions, clothing sizes for eBay product match, serial number recognition, etc.

##### A quick note on feature engineering
_These are all important points regarding __feature engineering__, and the main reason why we said earlier that developing a good understanding of the data is so important in NLP: unless you are certain that your data contains enough examples to account for the phenomena you want to model (so that enough features can be derived from it and the system can learn the appropriate behavior), the algorithm will not be able to make sense of the data. You cannot train a system for detecting flight information without information about flights. NLP often has the responsibility of keeping things real in machine learning and asking questions like **"So you want your system to detect funny content... Are you detecting any jokes right now?"**_ :)

## Letter case
The previous examples also contain a particular type of word variant, __letter case__. Case refers to whether a word is __UPPERCASE__, __lowercase__, __Titlecase__, or some __CoMbInAtIoN__ of both (the last one is often called __camel case__).

A common type of preprocessing step consists in lowercasing all words, collapsing or case variants into their lowercase form. This will normally preserve most of the original information (the meaning of _combination_ does not usually change based on its case), although there are obviously exceptions (a _bond_ refers to a type of agreement or to a strong link, whereas _Bond_ is a secret agent. In most corpora there will be no confusion, but that might not be the case in a dataset about movie reviews. Again, knowing your data is important to make this kind of decisions).

The goal of collapsing all these variants into a single one is to prevent the system from considering _the, The_ and _THE_ three different words, which would result in incorrectly dividing their frequency counts three-fold when calculating their probabilities, for instance. This type of normalization can easily boost the frequency of some terms several times, which results in a statistical model much closer to the actual truth of the data.

## On the topic of variants
Letter case is just a subtype of the more general fact that words have variants, and there are other types of variation that will make your NLP lives miserable. The two main types are __inflectional variation__ and __misspellings__.

Inflectional variation is what the type of variation whereby we recognize the words _child_ and _children_ to be two __forms__ of the same __lemma__. The lemma is usually seen as referring to the actual concept, whereas the forms account for morphological variation, alternative shapes that the lemma takes to fit in particular linguistic contexts such as the plural number, or verb tenses. In languages like Spanish or Ukrainian, inflections are much richer than in English, and the amount of variation is also much higher, which usually requires some additional processing.

In NLP, the two main ways of dealing with __forms__ is to either __stem__ them or __lemmatize__ them. Stemming is usually faster, easier, and error-prone. The main problem is that there is no guarantee that you end up with actual lemma. Lemmatization, on the other hand, tries to fulfill that requirement, but it has a number of relatively strong pressupositions: 1) part-of-speech tagging, which is not always trivial and that we will introduce later on, and 2) most often, a language-specific dictionary with mappings between forms and their lemmas. For now, let's take a look at the differences with a simple example. As always, NLTK can help us, although there are now other libraries we can use for this: __pattern__ and __spacy__, which offer better performance than NLTK for some tasks.

In [ ]:
#    Using NLTK
import nltk

#   stemmers
stemmer1 = nltk.PorterStemmer()
stemmer2 = nltk.LancasterStemmer()
stemmer3 = nltk.SnowballStemmer(u'english')
lemmatizer = nltk.WordNetLemmatizer()
lemmatizer.stem = lemmatizer.lemmatize
normalizers = [
    ('porter_stemmer', stemmer1),
    ('lancaster_stemmer', stemmer2),
    ('snowball_stemmer', stemmer3),
    ('wordnet_lemmatizer', lemmatizer)
]

test = 'Strange listening women lying children government distribution basis supreme power derives adequate ceremonies'

for name, normalizer in normalizers:
    print name, '\t=\t', [normalizer.stem(token) for token in tokenizer(test)], '\n'

In [ ]:
#    Using pattern
import pattern
from pattern.en import parse

#    Pattern is unnecessarily powerful at this point: its main output are parses,
#    complete syntactic analysis. We still don't need that so we will implement a
#    wrapper that returns the lemmas only:

class PatternStemmer:
    def __init__(self):
        return
    
    def __call__(self, text):
        tree = parse(text, lemmata=True)
        return [
            part[-1] for part in tree.split()[0]
        ]

stemmer = PatternStemmer()
print 'pattern_lemmatizer\t=\t', stemmer(test)

In [ ]:
#    Using spacy
import spacy

en_nlp = spacy.load('en')
en_doc = en_nlp(test.decode('utf-8'))

print [en_doc.vocab[x.lemma].orth_ for x in en_doc]

__spacy__'s lemmas look pretty great, actually, and are the best of all systems (for this particular example). Although we have skipped it here, __spacy__, like __pattern__, provides lemmas integrated into the syntactic analysis, which as we saw earlier is the only way of being able to choose the correct lemma: lemmas can differ based on a word's part-of-speech, which in turn must be determined by looking at its context in the sentence.

## Parts of speech
Now is probably a good time to introduce [parts of speech](https://en.wikipedia.org/wiki/Part_of_speech) (PoS for short). The basic idea comes from linguistics and it actually goes back many centuries. Latin grammarians (!!) already described word behavior based on parts of speech, which simply means how a word (form) behaves within a sequence of words.

To fully grasp parts of speech, it is better to think in terms of concepts:

> _person, height_

That is just a sequence of two ideas, two __nouns__. As such, there is no relationship between the two.

At this point, language is still not very informative: we are just naming things and all speakers are supposed to know the names of most things anyway. Nouns are a prerequisite for linguistic behavior, but not really its end goal. Language starts to be really useful when it __adds knowledge__, when new information is added through combinations.

We started with two ideas, _person_ and _height_ -what if there is a connection between the two? What if that person is tall? _Height_ now becomes an attribute of _person_ (both ideas go together), and language will now use an __adjective__ to express that the two ideas in the sequence are not independent but are actually connected. That is, when an idea specifies another idea, it becomes an adjective (or behaves like one, such as the genitive case in Ukrainian or _de_-preposition phrases in Spanish):

> _tall person_

So far, we have seen words or combinations of words, but we are still not conveying thoughts. That is, we can build any sequence of nouns and adjectives, as long as we want. The question is, what for? Or rather, where do we stop? The standard way to mark that a sequence of ideas is a closed, complete unit that we can then transmit, is through a __verb__:

> _People grow (become taller until they reach adulthood)_    (~_stature_ sense of _height_)

> _People climb_                            (_vertical movement_ sense of _height_)

> _People (are) at the top_                 (_vertical_ position sense of _height_)

> Etc.

The examples above still involve the same concepts as before, yet only these last examples are valid sentences that we can find in a corpus, something with a truth value that can be regarded as an assertion about the world. Verbs turn potentially endless sequences of noun combinations into a bounded thought we can express.

These are the main parts of speech, although there are a few others:
* pronouns (placeholders for nouns),
* adverbs (adjectives for verbs or for other adjectives),
* prepositions (they create adverbs from nouns or noun phrases, including pronouns),
* conjunctions (they create nouns, adjectives or adverbs from verbs and their nouns),
* determiners and quantifiers (usually adjectives in origin, but they end up as an indispensable part of nouns, to the point that nouns cannot be used without some determiner. For example, countable nouns in the singular can almost never be used without a determiner: _The dog barked_ versus _*Dog barked_).

The three first categories (noun, adjectives and verbs) are open classes: new words are being added every day (especially true for nouns, just think of all new movie and song titles, people names, software version names, etc.) The last four are usually closed classes and their elements tend to remain constant over very large time spans.

This is a very broad and overly simplified overview of parts of speech. Since language is messy, there are often cases of unclear boundaries across categories, as well as many cases of ambiguous words intersecting several categories. We will go back to this later.

## Stopwords (or rather, _high-frequency noise_)

As it turns out, words belonging to closed classes are those that are usually considered __stop words__. In the field of Linguistics, the words in those classes are often called __function words__ because, rather than conveying meaning, they often perform some more abstract, functional role inside the sentence (that is, they replace open-class words or change their part of speech).

In many NLP applications, it is not necessary to go too deep into the linguistic structure of a sentence, which means that the system does not need to understand stop words. Sometimes they even hurt performance: they have extremely high probabilities that can easily mislead a naïve statistical system.

For this reason, stop words are very often removed as part of the text preprocessing step.


### Long-tail distributions
Natural language vocabulary is well-known for exhibiting a pattern corresponding to [Zipf's law](https://en.wikipedia.org/wiki/Zipf%27s_law), which is a subcase of the more general [long tail](https://en.wikipedia.org/wiki/Long_tail) statistical distribution (long-tail distributions [have a well-known shape](https://www.dropbox.com/s/ev0r95z8o9iahxc/Long%20tail.png?dl=0)), whereby a small number of outcomes have high frequency and a large number of outcomes have a very low frequency. In language, however, this will not be likely to result in a standard [power law](https://en.wikipedia.org/wiki/Power_law) distribution. Zipf went probably too far when he stated that the frequency of the word at rank _i_-th is twice that of the word at rank _i + 1_-th, but his general point still applies:

In [ ]:
from collections import Counter

from lib.TextStreamer import TextStreamer

class ZipfsLaw(Counter):
    
    def __init__(self):
        return
    
    def total(self):
        return float(sum(zip(*self.most_common())[1]))
    
    def half(self):
        acc = 0
        t = self.total()
        for i, (w, f) in enumerate(self.most_common()):
            acc += f
            if acc / t >= 0.5:
                return i + 1
        return i

                
path = '/Users/jordi/Laboratorio/corpora/raw/umbc/webbase_all/delorme.com_shu.pages_89.txt'
strr = TextStreamer(path)
i = 0
zipf = ZipfsLaw()
for record in strr:
    zipf.update([w.lower() for w in tokenizer(record.strip()) if w.isalpha()])

print zipf.most_common(10)
print zipf.total()
print zipf.half()

The words in the top-10 above are __all__ stop words (function words). In Zipf's original experiments, 135 words were enough to account for half of the probability mass of the [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus). We see that this number remains largely the same even today, and that the number is still very close in our case: 144 words account for half the probability mass of our dataset.

In some way, there is a constant core of what makes a language the way it is, and we can expect that to be true of any dataset. Of course, this also means that any dataset will contain at least these words and, based on these words, any dataset of a language will "match" any other dataset of that language to some extent, making classification tasks hard (there is always some similarity between two English texts: English).

This is the central idea behind applications like [__language detection__](https://en.wikipedia.org/wiki/Language_identification), but other NLP tasks aiming at a higher resolution need a way to ignore these common linguistic core.

The first way anyone ever thought of doing that, was to create a list with those ~150 words (more or less) and use it as a blacklist for the system: any word in that list would be removed from the document's tokens, otherwise they stayed. As usual, we will rely on NLTK for a well-maintained list of stopwords:

In [ ]:
from nltk.corpus import stopwords

from collections import defaultdict as deft

blacklist = deft(bool)
for w in stopwords.words(u'english'):
    blacklist[w] = True

new_top_10 = [(w, f) for w, f in zipf.most_common() if not blacklist[w]][:10]
print new_top_10

It is definitely better (words such as _work, story_ or _book_ now have reached the top-10), but we still have a few words that do not tells us much: _one, time, new_ or _would_. These words can be considered high-frequency noise: words that may be actual words (rather than __function words__) strictly speaking, but that still convey too little information anyway to be helpful in our applications.

However, we have already used the stopword-list trick. Now what? This is where things start to get interesting. 

### Dealing with high-frequency noise

At some point, people realized that having to tell the system which words are important and which ones are not, is not only boring for us, but also does not say much of our NLP skills: is it not possible to find a way of automating this? As it turns out, there is :)

One could argue that, if our goal is to develop a really smart machine, something that gets close to artificial intelligence in any meaningful sense,  then that kind of system should also be able to figure out on its own what is important from what is not (to at least a reasonable extent). Although that claim is a bit vague, we can make it more specific by saying that we want the system to be able to learn on its own at least which words convey too little information to be meaningful within a particular dataset.

There are many ways of doing that but we will take a look at a few of the most effective: __background-foreground overlap, TF-IDF__, and __conditional probability__ (important because we will be using it in the context of classifiers like Naive Bayes).

#### Background-foreground overlap
The idea here is simple, we take a frequency distribution for one dataset, another frequency distribution for another dataset (totally independent, or as unrelated as possible), look at the top _n_ words in each distribution, and remove any items in the intersection. We will be left with much more informative content.

In [ ]:
import nltk
from nltk.corpus import reuters

from collections import Counter
from lib.Tools import tokenizer

fg_sents = [tokenizer(record) for record in strr]
fg_dist = Counter()
for sent in fg_sents:
    fg_dist.update([w.lower() for w in sent if w.isalpha()])

bg_dist = Counter([w.lower() for w in reuters.words() if w.isalpha()])

topn = 500

bg_top = [w for w, _ in fg_dist.most_common(topn)]
fg_top = [w for w, _ in bg_dist.most_common(topn)]

noise = [w for w in fg_top if w in bg_top]
content = [w for w in fg_top if w not in bg_top]

print 'noise:', len(noise), noise[:10]
print 'content:', len(content), content[:10]
print 'known stopwords:', len([w for w, boolean in blacklist.items() if boolean])
print 're-used stopwords:', len([w for w in fg_top if w in bg_top and blacklist[w]])

#### TFIDF (mutual information) and conditional probability
TFIDF stands for __term frequency * inverse document frequency__. The intuition behind it is that the most relevant words in any document will be the most frequent words in that document (because they appear all over the place: "work, work, work") that are, at the same time, the least frequent on the overall dataset (in the sense that they will be unique to the current document: if no or very few other documents speak about the same topic, that topic can be assumed to be very characteristic of that particular document). So, it is a mathematical formula of what is both __important (frequent) and specific (inversely frequent over all documents)__ about a given document. This type of information is usually called the most __salient__ information.

TFIDF often works at the document level within the same dataset, so here the system in fact needs the dataset to be divided into documents (that means that TFIDF cannot be directly calculated for short text inputs such as tweets, or at least that it will not yield any meaningful improvements over raw frequency counts unless some other segmentation of the data can be provided, at the topic level for instance).

Fortunately, the Reuters corpus is divided into documents so we can use it for our experiments. It is no coincidence ;)

The TFIDF formula is quite popular in the field of Information Retrieval and it has been successfully applied in many NLP applications. It is often a standard measure of term relevance and you can hardly go wrong if you use it. Sometimes it seems that half of all NLP happens around the TFIDF formula :) (or some variation of it).

In [ ]:
import math
from collections import defaultdict as deft


#    The number of documents in our corpus:
N = len(reuters.fileids())
print N


#    The TFIDF formula:
def tfidf(word, frequency, document_frequencies):
    tf = frequency
    idf = math.log(N / float(document_frequencies[word]), 10)
    return tf * idf


#    First pass over the data to collect the document frequency for each word:
document_frequencies = Counter()
for i in reuters.fileids():
    doc = reuters.words(i)
    for w in set(doc): 
        document_frequencies[w] += 1

        
#    Second pass over the data to calculate the TFIDF scores for the words
#    in each document:
for i in list(reuters.fileids())[:5]:
    doc = reuters.words(i)
    counts = Counter(doc)
    print 'top-5 by frequency:\t', counts.most_common(5)
    tfidf_weighted = sorted(
        counts.most_common(),
        key=lambda x: tfidf(x[0], x[1], document_frequencies),
        reverse=True
    )
    print 'top-5 by TF-IDF:\t', tfidf_weighted[:5]
    print 

#### Conditional probability

__Conditional probability__, on the other hand, is based on a slightly different idea: for conditional probability, the most important words in a document are not necessarily the most frequent ones.

For conditional probability, what makes a word relevant is how much its probability changes in a given subsample of the data with respect to its overall probability over the entire dataset. So, rather than simply taking the most frequent words in a document that are not high-frequency noise (as TFIDF does), conditional probability focuses on the words whose probability changes the most, regardless of how frequent they are.

Sometimes, this results in more informative features for our system: whereas TFIDF is really good at modelling topical information about a document, it may also overlook relevant specificities about that document. Let's see it with an example -the class ProbDist below will calculate for us both __TFIDF__ and __conditional probability__ scores for the same documents, and display the top-5 words according to each metric:

In [ ]:
import nltk
import math
from collections import (
    Counter,
    defaultdict as deft
)

from nltk.corpus import reuters


import nltk
import math
from collections import (
    Counter,
    defaultdict as deft
)

from nltk.corpus import reuters


class ProbDist:

    def __init__(self):
        self.priors = Counter()
        self.mass = 0.0
        self.classf = deft(float)
        self.posteriors = deft(Counter) 
        self.masses = deft(float)
        self.idf = deft(float)
        self.n = 0

    def update(self, i, tokens):
        l = len(tokens)
        self.classf[i] += 1
        self.posteriors[i].update(tokens)
        self.masses[i] += l
        self.priors.update(tokens)
        self.mass += l
        self.__idf_update(i, tokens)
    
    def __idf_update(self, i, tokens):
        for w in set(tokens):
            self.idf[w] += 1
        self.n += 1
    
    def most_common(self, i, n=None):
        return self.posteriors[i].most_common(n)
    
    def most_cond(self, i, n=None):
        pxy = [(w, self.prob(w, i)) for w, f in self.posteriors[i].items()]
        px_y = {w: self.prob(w) for w, _ in pxy}
        conds = sorted(
            [(w, (math.log((p / px_y[w]), 2))) for w, p in pxy],
#             [(w, p * (p / px_y[w])) for w, p in pxy],
            key=lambda x: x[1],
            reverse=True
        )
        if not n:
            n = len(conds)
        return conds[:n]
    
    def tfidf(self, i, w):
        tf = self.posteriors[i][w]
        idf = math.log(self.n / self.idf[w], 10)
        return tf * idf
    
    def most_tfidf(self, i, n=None):
        tfidfs = [
            (w, self.tfidf(i, w))
            for w, _ in self.posteriors[i].items()
        ]
        tfidfs.sort(key=lambda x: x[1], reverse=True)
        if not n:
            n = len(tfidfs)
        return tfidfs[:n]
               
    def prob(self, w, i=None):
        if i:
            return self.posteriors[i][w] / self.masses[i]
        else:
            return self.priors[w] / self.mass
    

#    First pass over the data to collect prior probabilities:
prob_dist = ProbDist()
for i in reuters.fileids():
    doc = reuters.words(i)
    prob_dist.update(i, doc)

        
#    Second pass over the data and calculation of posterior probabilities:
for i in list(reuters.fileids())[:5]:
    print reuters.categories(i)
    print 'top-5 by frequency:\t', [w for w, _ in prob_dist.most_common(i, 5)]
    print 'top-5 by COND:\t', [w for w, _ in prob_dist.most_cond(i, 5)]
    print 'top-5 by TFIDF:\t', [w for w, _ in prob_dist.most_tfidf(i, 5)]
    print 

As you can see, __conditional probability__ scores higher slightly more specific (less frequent) words that are still relevant to the topic, such as _Sumatran_ in the last document (Sumatra being a part of Indonesia), _additives_ in the second one (definitely relevant in the context of food _preservation_) and _kilowatt, kilolitres_ and _kl_ in the third one, which all make sense in a document about energy production.

As we can see, __TFIDF__ and __conditional probability__ complement each other nicely, in the sense that the former is good at describing the general topic of a document, whereas the latter is good at spotting strongly associated terms that can be used for a more fine-grained semantic analysis of the document's information.

Incidentally, this is also good to spot potentially important features in our dataset. Remember how we said earlier that, sometimes, it is up to us to decide if our system should take numerical expressions into account or not? (the flight-booking website example). In these examples we are seeing the system trying to find out on its own :)

The _kilowatt_ word (and a potential numerical expression nearby associated to it) may not be the most frequent word in a document about energy (and TFIDF may therefore tend to not see), but it is still very salient when measured using conditional probability (because, even if it appears only a few times in the document, it appears even more rarely in the entire dataset). That means that it is probably a good predictor of the class (every time the system finds the word _kilowatt_, it can classify the docuemnt as being about energy) and an equally good feature for our system.

__Tip:__ it follows that, if we multiply the conditional probability of a word by its posterior probability in a document, we get a value which is nearly identical to its TFIDF.

# BOWs

We have now reached a crucial point: after everything we have seen, we not only have a number of ways of breaking down a document into its constituent parts beyond simple tokens, but we also have metrics to approximate an idea of content relevance that is within the range of cognitive plausibility[1], what we as human perceive as relevant ourselves.

This means we already have a granular, more discrete representation of a text in terms of dimensions that we can use to measure it, as well as quantitative measurements along those dimensions using the metrics we introduced. Put in other way: until now, all we had was an area. Now we have axes along the dimensions of that area that we can use to place coordinates and locate specific points inside that area.

We are finally in a position to start working with text in a scientific, formal way. And if its formal, the a computer can understand it :)

The representation that we have been introduced in the previous sections is known as the __Bag-of-Words (BOW)__ model. It is based on the metaphor of our space as a bag in which words are placed, just like [marbles](https://www.dropbox.com/s/bf62o5jljma6ezd/Marbles.jpg?dl=0) :D (Yep, illustrating that was absolutely necessary ;)

Each document is a bag of marbles, sorry, __words__ :) and we can assess how similar two documents are by comparing the marbles they contain, sorry, __words__. Oversimplifying __a lot__, it is possible to say that this is the main idea behind most __search engines__, in terms of how retrieve documents from an index.

Actually, let's try that ourselves: let's pick a random document from the Reuters corpus and let's display the top-5 most similar documents, to see if we just hacked our own home-made Google. First, we'll do it with simple Python tools, then we will learn the proper way of doing it ;)

[1] __Plausible__: something that makes sense to a human observer, or something that gets results using a method that can be expected to be close to the actual phenomenon generating those results. We can explain a result of 2 as either _1 + 1_ or _(6 + 3) - (6 + 1)_. The first option is the most plausible (most reasonable). In this sense, the notion of plausibility has strong ties with notions like [Occam's razor](https://en.wikipedia.org/wiki/Occam%27s_razor) and [theoretical elegance](https://en.wikipedia.org/wiki/Elegance):
> The proof of a mathematical theorem exhibits mathematical elegance if it is surprisingly simple yet effective and constructive; similarly, a computer program or algorithm is elegant if it uses a small amount of code to great effect.


In [ ]:
#    Home-made search engine using BoWs -the "hack way"

#    First, we build the index of documents by their words
#    and compute their BoWs at the same time:

documents_by_word = deft(list)    # We will use an inverted index to speed up lookup

bows = dict([])
for i in reuters.fileids():
    bow = dict([])
    for word, weight in prob_dist.most_tfidf(i):
        bow[word] = weight
        documents_by_word[word].append(i)
    bows[i] = bow


#    After that, we iterate over the documents again
#    and retrieve the most similar documents:

#    A function to get all BoWs sharing some word
#    with the input BoW:
def documents_with_some_word_in_common(bow, top=None):
    if not top:
        docs = set([])
        for w in bow.keys():
            docs.update(documents_by_word[w])
        return docs
    else:
        docs = Counter()
        for w in bow.keys():
            docs.update(documents_by_word[w])
        return set([doc for doc, f in docs.most_common(top)])


#    A function to remind us of the top-5 words
#    by TFIDF in a particular BoW:
def top5(bow):
    return sorted(
        bow.items(),
        key=lambda x: x[1],
        reverse=True
    )[:5]


#    A function to calculate the similarity
#    between two BoWs, for now simply as the
#    ratio between the TFIDF of all words
#    shared between the two, and the total
#    TFIDF of all their words, averaged, and
#    minus the average of difference between
#    the two:
def similarity(bow, i):
    sim = 0.0
    _sim = 0.0
    _bow = bows[i]
    shared_words = set(bow.keys()).intersection(set(_bow.keys()))
    total_tfidf = sum(bow.values())
    _total_tfidf = sum(_bow.values())
    matches = []
    for w in shared_words:
        sim += bow[w] / total_tfidf
        _sim += _bow[w] / _total_tfidf
        matches.append((w, bow[w]))
    return (((sim + _sim) / 2) - (abs(sim - _sim) / 2),
            sorted(matches, key=lambda x: x[1], reverse=True)[:5])


#    A function to see the first 20 words of a
#    particular Reuters docuemnt:
def txt(i):
    return ' '.join(list(reuters.words(i))[:100])



for i, bow in bows.items()[:5]:
    candidate_bows = documents_with_some_word_in_common(bow)
    sim_bows = sorted(
        [(candidate, similarity(bow, candidate), txt(candidate))
         for candidate in candidate_bows],
        key=lambda x: x[1][0],
        reverse=True
    )
    print i
    print txt(i)
    print reuters.categories(i)
    print '---'
    for candidate, sim, text in sim_bows[1:4]:
        print '  ', candidate
        print '  ', reuters.categories(candidate)
        print '  ', sim
        print '  ', text
        print '  ', '----'
    print '====='
    print

There are several cool things going on here:

1. Highly-relevant, topically-related matches for the first group about company acquisitions.
2. It goes deeper than that, though -the first example is also an instance of a thoroughly correct category prediction __just by adding up TFIDF weights over an inverted index__.
3. The same considerations apply to the second and third groups, on the crude industry and the consumer price index, respectively.
4. Something even cooler can be observed in the fourth group: a machine prediction that __goes beyond__ the human annotation, that is, __an actual instance of machine learning__ :) The original document is about trade disputes (_trade_ category), but the system is matching to two other documents in the _trade_ category __and__ a document belonging to the _corn, grain_ categories. When we take a look at the document, however, we can see it is in fact about **trade disputes in the _corn, gran_ industry**. The system has been able to make a correct inference and assign a new, correct category to the document, even when the human annotators did not do that themselves. This is a perfect example of bootstrapped/semi-supervised learning: we can first train an algorithm on some data annotated by people, and then use that system's decisions to easily add more data to our initial pool. Once a system gets to the point of being useful, it quickly becomes unstoppable :)
5. And the cooler-yet result is hidden in the last group: at first, it seems a like a weird dump of different categories. All categories appear only once: the original document labeled as _earn_, the first suggestion as _iron-steel_, the others as _crude_ and _copper_. So, what happened here? Again, __machine learning happened__ :) If we pay attention to the documents' title, we can see that, in this group specifically, all dcuments start with the _TALKING POINT_ phrase. These documents are reports by analysts and feature prominently quotes of their statementsl. Although they have not been explicitly labeled as such in the Reuters corpus because in that corpus the tags are topic-based rather than genre-based, the algorithm does __sense__ something special and consistent about this group. If we look at the features triggering each match, it becomes evident that the pronoun _he_ is a constant, which makes sense because articles quoting analysts' opinions will tend to have a significant number of expressions like _he said, he claimed, he reported_. The system is able to pinpoint that on its own, and it is also interesting to think what would have happened if we had removed all stop words (_he_ is almost always considered one) -we would have lost one of the main predictors for the _talking point_ subset of our sample and would have probably never known it was there. Again, knowing the data and trying to minimize our assumptions on its nature proves to be crucial for modelling linguistic phenomena correctly.

All things considered, we are already getting plausible results :) Or, as one of my favorite [role models](https://en.wikipedia.org/wiki/Ash_Williams) would, **_groovy_** results :) [1].

[1]
> fashionable and exciting: sporting a groovy new haircut.

> enjoyable and excellent: he played all the remarkably groovy guitar parts himself.


Ok, now let's stop playing and let's start taking things seriously. What is the proper way of doing that? The main difference is how the similarity is calculated (over all the dimensions in the dataset), and it involves __vectorization__, turning our BoWs into [__vector__](https://en.wikipedia.org/wiki/Vector)s. 

> in pattern recognition and machine learning, an n-dimensional vector of numerical features that represent some object

Vectors are a very popular input representation and most algorithms expect it or can at least handle it. When you leave home, you should always take with you some code to vectorize your data :)

The thing with vectors is, they can grow really big :D Particularly for high-dimensional data like natural languages, where each word is viewed as one dimension and that means that each vector will be __as long as the entire vocabulary__ in our dataset (often referred to as the [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality)), and most of its values will be __zeroes__ (often referred to as [data sparsity](https://en.wikipedia.org/wiki/Sparse_matrix))[1].

[1] By the way, matrices of BoWs are usually called __vector space models__, which is a cool name and a buzz word you should try to throw around whenever you have a chance ;) To make it sound even more impressive, you can also add the adjective _semantic_ up to a grand total of __semantic vector space models__. Apart from the cool factor of the name, it also happens to be technically correct :) (since we are building as our model a space containing some number of vectors, and the information in those vector faithfully represents semantic information -the relevance we saw in earlier sections-).

That is why more efficient representations are commonly used. However, vectors are still at the core of any such representation, which simply use some trick or other to transform the vector into a more manageable object __without information loss__. Things can get very complicated in this area and there is an entire field of [dimensionality reduction](https://en.wikipedia.org/wiki/Dimensionality_reduction) which is absolutely fascinating but totally beyond the scope of this course :D (For those who are really curious, you can check out (Random Forests/Ensemble Trees)[https://en.wikipedia.org/wiki/Random_forest] or (Principal Component Analysis)[https://en.wikipedia.org/wiki/Principal_component_analysis].

To keep things simple, we will repeat the previous experiment using Python's standard Array object from the __numpy__ library. We will convert our BoWs into vectors, then calculate the similarity between documents using the __cosine similarity__. Wait, what?

A detailed technical explanation of intuitions behind the [__cosine similarity__](https://en.wikipedia.org/wiki/Cosine_similarity) would take some time :) so, to keep things simple, let's just say that it calculates the ratio between the sum of the weights of any non-zero-weight dimensions shared by two vectors, and the combined total weights for all dimensions of those two vectors. Because of operating in an Euclidean space, the final calculation also involves the __dot product__ and the __square root__ (see below for details :)

In [ ]:
import math, numpy, scipy


A = numpy.array


#    Re-used from: http://stackoverflow.com/q/3121217
#    Python's scipy or numpy implementations would usually be
#    the way to go but they expected unnecessarily complicated
#    wrapping objects as input and implementing the function
#    to handle unwrapped vectors will be much easier in our case.
def cosine_distance(u, v):
    """
    Returns the cosine of the angle between vectors v and u. This is equal to
    u.v / |u||v|.
    """
    return numpy.dot(u, v) / (math.sqrt(numpy.dot(u, u)) * math.sqrt(numpy.dot(v, v))) 
cos = cosine_distance


#   An object necessary to build our vector matrix,
#   a mapping from words to their positional IDs in
#   the vector space:
dimensions = dict([])
i = 0
for w in set(reuters.words()):
    dimensions[w] = i
    i += 1

D = len(dimensions.keys())


#    Function to transform a our current BoW objects
#    (Python dictionaries) into vectors:
def bow2vec(bow):
    vector = [0.1 for i in range(D)]
    for w, weight in bow.items():
        if not known_word[w]:
            continue
        word_index = dimensions[w]
        vector[word_index] = weight
#     print [x for x in vector if x]
#     return A(vector).reshape(1, -1)
    return A(vector)


#    Home-made search engine using BoWs -the cool way (with vectors)
for i, bow in bows.items()[:5]:
    
    candidate_bows = documents_with_some_word_in_common(bow, top=50)
    
    sim_bows = []
    for candidate in list(candidate_bows):
        vec1 = bow2vec(bow)
        vec2 = bow2vec(bows[candidate])
        sim = cos(vec1, vec2)
        sim_bow = (candidate, sim, txt(candidate))
        sim_bows.append(sim_bow)

    sim_bows = sorted(
        sim_bows,
        key=lambda x: x[1],
        reverse=True
    )

    print i
    print txt(i)
    print reuters.categories(i)
    print '---'
    for candidate, sim, text in sim_bows[1:4]:
        print '  ', candidate
        print '  ', reuters.categories(candidate)
        print '  ', sim
        print '  ', text
        print '  ', '----'
    print '====='
    print

It looks good :)

1. The company acquisitions topic is again matched to several other documents in the same category effortlessly, by the very logic of their content.
2. Same for the crude industry topic, which is also even more topically consistent, with all documents being now about news on Iran, just like the original news item.
3. The group for the consumer prices index also seems semantically consistent but here it seems that our hacky way of solving this problem may have performed slightly better :) Or simply managed to stay closer to the manual annotation of the data for whatever other reason. In any case, both systems yield plausible results.
4. Same for the last topic, the one about TALKING POINTs: again, cosine gets close to finding the hidden category but it would seem again that our home-made implementation has been a bit more sensitive to this particular variable.

So, these are the foundations of the BoW/(semantic) vector space model as a representation of natural language inputs.

# Classification

We have features, we have BoWs, and we know how to turn them into vectors. It is time to put everything together and start predicting. Prediction is the key to automation: given all the previous pairs of _(X, Y)_ that our system has been trained on, the system must now be able to predict a correct _Y_ for a new, previously unobserved _X_.

Prediction is performed by building a model: a weighted mapping from features to classes. The classifier will learn how well each feature predicts a given class (that is, how much weight it provides for the hypothesis that _X, therefore Y_).

Classifiers differ in the way in which they compute the weights, that is, in which specific algorithm they use for weighting the features. There are many types of algorithms to do that but, in the field of NLP, some of the most important ones are [Naïve Bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) classifiers, [Maximum Entropy](https://en.wikipedia.org/wiki/Multinomial_logistic_regression) (Logistic Regression) classifiers, and [Support Vector Machines](https://en.wikipedia.org/wiki/Support_vector_machine) classifiers.

We are going to try something crazy and understand each of them in a few lines :) (by the way, we will be borrowing some ideas from [these guys](https://www.datarobot.com/blog/classification-with-scikit-learn/), who provide a really nice introduction to some of the concepts and is definitely worth reading).

## Naïve Bayes

In [ ]:
#    Naive Bayes

#    Below are the features for our model -the known facts
#    we have observed in the data.
#
#    These are the columns of each row in the list of 
#    'facts' :
#
#      [0] Name of the class
#      [1] Stars at least one socially awkward geek
#      [2] Stars doctors
#      [3] Stars families of rich people
#      [4] Stars dwarfs and dragons
#      [5] Stars talking computers
#      [6] Total shows
#
facts = [
    ('good_TV', 40, 15, 10, 35, 50, 150),
    ('grown_up_TV', 10, 60, 20, 9, 1, 100),
    ('bad_TV', 5, 5, 189, 1, 1, 200),
    ('internet_TV', 100, 5, 20, 22, 3, 300)
]


class Category:    
    def __init__(self, row):
        self.posteriors = []
        self.frequencies = []
        self.name = row[0]
        self.mass = float(row[-1])
        for value in row[1:-1]:
            self.posteriors.append(value / self.mass)
            self.frequencies.append(value)
    
    def __call__(self, features_in_show):
        probs = [
            self.posteriors[i] * feature
            for i, feature in enumerate(features_in_show)
        ]
        if not probs:
            return 0.0
        prob = probs[0] + (10 ** -4)
        for _prob in probs[1:]:
            if _prob:
                prob *= _prob
            else:                        #   Simple additive smoothing to avoid zero-multipli-
                prob *= (10 ** -4)       #   cations, which would cancel out our probability 
        return prob                      #   estimates. This is a well-known drawback of Naïve
                                         #   Bayes and there are a number of techniques for 
                                         #   performing statistical smoothing:
                                         #   https://en.wikipedia.org/wiki/Smoothing
        
class Model:

    def __init__(self, facts):
        self.categories = [Category(row) for row in facts]
        self.mass = float(sum(c.mass for c in self.categories))
        self.class_priors = {
            i: c.mass / self.mass for i, c in enumerate(self.categories)
        }
        self.feature_priors = [0.1 for feature in facts[0][1:-1]]

        for c in self.categories:
            for i, value in enumerate(c.frequencies[1:-1]):
                self.feature_priors[i] += value


    def prior(self):
        prob = self.feature_priors[0]
        for _prob in self.feature_priors[1:]:
            prob *= _prob
        return prob
        
                
    def __call__(self, new_show):
        features_in_show = new_show[1:-1]
        hypotheses = []
        for i, c in enumerate(self.categories):
            pxy = c(features_in_show) * self.class_priors[i]
            px_y = self.prior()     # This part does not really affect the calculations
            p = pxy / px_y          # because it remains constant for all classes.
            hypotheses.append((p, c.name))
        return sorted(hypotheses, reverse=True)
    

#
#     [1] Stars at least one socially awkward geek
#     [2] Stars doctors
#     [3] Stars families of rich people
#     [4] Stars dwarfs and dragons
#     [5] Stars talking computers
#     [6] Total shows
#
m = Model(facts)
print m((None, 0, 0, 0, 1, 1, None)), '\n'
print m((None, 0, 1, 0, 0, 0, None)), '\n'
print m((None, 0, 1, 1, 0, 0, None)), '\n'
print m((None, 0, 0, 1, 0, 0, None)), '\n'
print m((None, 1, 0, 0, 1, 0, None)), '\n'

#### The independence assumption
Why is __Naïve Bayes__ called "naïve"? Because the model makes an assumption that is usually too well-intentioned :) Naïve Bayes models work on the feature __independence assumption__, which is, by the way, the mean reason why they are simple and fast, but also the main reason why so many people wonder how they can perform so well most of the time :)

In NLP, the __independence assumption__ translates into the (very strong) assumption that the probabilities of each word in the document are independent from the presence of any other word in that document. That is, Naïve Bayes calculates the probability of words given the category of the documents they appear in (for instance, the fact that the word _politician_ appears more frequently in texts about politics), but it does not take into account the probability of a word given some other word (for instance, the fact that _politician_ and _ministry_ tend to appear together, or __co-ocurr__,  using the technical term, which stands for "occurring/appearing together").

For Naïve Bayes, this means that two correlated words will each contribute to the overall score of the class. For the example above, _politician_ and _ministry_, that seems fine, though, since a text containing two words associated with the politics category should get some score from each separate word anyway. Independence seems fine in this case.

When is it not okay? In NLP, the __independence assumption__ can be particularly harmful with __collocations__ and __multiwords__. We will see these units in more detail in a bit, for now let's just say they are terms consisting of several tokens, such as _Barak H. Obama, president Barack Obama, cafe latte_ or _Lviv Computer Science Summer School_.

From Naïve Bayes naïve perspective, each of the tokens in each of these words would add a proportional probability. That means that documents _d1_ and _d2_ below would both receive the same probability 

> training_doc1 = {president, Barack, Obama, trade, deal}

> training_doc2 = {Obama, war, Iran}

> ------

> new_doc = {president, Barack, Obama, war, Iran}

> ------

> training_doc1 ∩ new_doc = 3 / 5

> training_doc2 ∩ new_doc = 3 / 5

despite the fact that 'training_doc2' is clearly the most relevant given the new document. To summarize, the __independence assumption__ may cause result in false positives due to awarding free (too generous) hits for tokens that are really parts of just one word. In NLP, this is a direct consequence of work-in-progress tokenization (now you see the importance of all that boring stuff about tokenization I told you about earlier ;) and how Computational Linguistics can make things easier for Machine Learning -__this__ is NLP). What this means is that not everything you can split on a whitespace is a word (and the fact that the opposite is also true is a *__well-known__ fact* ;), and sometimes the actual linguistic unit (the thing whose probability you should be calculating) will be something bigger, sometimes spanning a few words and sometimes even *discontinuous*, which makes it necessary to *__take__ these things __into account__*.


## Logistic Regression
On the other hand, __Logistic Regression__ or __Maximum Entropy__ (usually abbreviated as __MaxEnt__) classifiers work by finding linear correlations between features (also called __independent variable__s, or __predictors__) and classes (also known as __dependent variable__s). Linear regression is the type of analysis that can tell you if people tend to have increasing amounts of ice-cream as temperatures rise towards the summer.

The key difference of logistic regression with respect to standard linear regressions is that, whereas linear regression gives us a coefficient (how associated the independent and dependent variables are, with a value ranging from -1 to +1), logistic regression gives us a __probability-looking normalized score__ (a floating point number between 0. and 1.0) that we can use as an actual probability of the class (dependent variable) given the feature (independent variable). Zero means the feature does not predict the class, one means the feature predicts the class with absolute certainty. The normalization of this score is performed using the [__logistic function__](https://en.wikipedia.org/wiki/Logistic_function) (a type of [sigmoid function](https://en.wikipedia.org/wiki/Sigmoid_function)) hence the technique's name.

The math behind logistic regression is a bit complicated and beyond the focus of this course. For convenience, below is some code ([via StackOverlow](http://stackoverflow.com/a/7939259)) with a recap of linear regression. For our purposes, imagine that we replaced [a] temperatures over a period of time and [b] ice-cream in our example above, with [a'] words in a document and [b'] document categories, respectively.

In [ ]:
facts = [
    # category, 'government', 'politician', 'player', 'team'
    ('politics', [1, 1, 0, 0]),
    ('politics', [0, 1, 0, 0]),
    ('politics', [1, 0, 0, 0]),
    ('sports', [0, 0, 1, 1]),
    ('sports', [0, 0, 0, 1]),
    ('sports', [0, 0, 1, 0]),
]

categories = [
    ('politics', [2, 2, 0, 0]),
    ('sports', [0, 0, 2, 2])
]

def average(x):
    assert len(x) > 0
    return float(sum(x)) / len(x)

def correlation(x, y):
    assert len(x) == len(y)
    n = len(x)
    assert n > 0
    avg_x = average(x)
    avg_y = average(y)
    diffprod = 0
    xdiff2 = 0
    ydiff2 = 0
    for idx in range(n):
        xdiff = x[idx] - avg_x
        ydiff = y[idx] - avg_y
        diffprod += xdiff * ydiff
        xdiff2 += xdiff * xdiff
        ydiff2 += ydiff * ydiff
    return diffprod / math.sqrt(xdiff2 * ydiff2)

new_doc = [1, 0, 0, 0]

for name, vector in categories:
    print name, correlation(vector, new_doc)

In [ ]:
#    And then the logistic function:
def logistic_function(v):
    return 1 / (1 + (math.e ** -v))

print logistic_function(0.57735026919)
print logistic_function(-0.57735026919)

I think we just implemented an again-grossly-oversimplified Maximum Entropy classifier :)

#### Back to the naïveté of correlations
The key advantage of __Logistic Regression__ over __Naïve Bayes__ lies in the former's ability to handle correlated features (check out this [thread on Quora](https://www.quora.com/What-is-the-difference-between-logistic-regression-and-Naive-Bayes) for details). That is, Logistic Regression __does not make the independence assumption__ and is able to deal feature correlations naturally by assigning them lower weights. In this way, a several-word-long linguistic unit no longer contributes to the final score for each of the tokens in it.

More specifically, Naïve Bayes calculates a probability for each word and finally multiplies the probabilities for all words, whereas Logistic Regression learns the weight for each word relative to the weights of rest of the words and, therefore, assigns to each its individual contribution over the maximum contribution that they all add up to.

That means that,iInstead of three words, each with an independent probability of 1.0 of belonging to the multi-word _president Barack Obama_, we now have three words, each with a ~0.33 probability of belonging to that multi-word, which makes far much more sense because the probability mass now sums up to 1.0 instead of 3.0.



## Support Vector Machines
__Support Vector Machines__ (SVMs for short), finally, are a powerful algorithm with occasionally slightly crazy training times. We mention them here because they are robust and very successful for some tasks, although their performance varies widely with the particular task. Essentially, they perform the same type of analysis as Logistic Regression classifiers (if we are talking about linear SVMs) with some differences in what is considered or high or low correlation.

In practice, Naïve Bayes often performs just as well as SVM classifiers (or better, despite their much higher simplicity) and Logistic Regression usually outperforms both. In my own experience, Maximum Entropy is usually the way to go.

The main factor to look for when deciding when to use SVMs is the dimensionality of our space: SVMs can handle high numbers of dimensions but they usually cannot handle the multi-dimensionality of NLP data (at least, no without some serious normalization and dimensionality reduction, which are always possible).

That is the main reason why they will probably not be very useful unless you have some way to carry out a massive clean-up and normalization of the data.

# Actual classifiers and actual data

We have everything we need: features, labels, BoWs, a vector space model, and classification algorithms to learn a mapping from the features to the labels. It is time to start doing some actual kick-ass prediction.

Let's put all the pieces together and go back down to the code once again:

### A dataset please

Or more than one if possible :)

In [12]:
import nltk
from nltk.corpus import reuters, brown

#    We will re-use the SimpleCorpusReader class we saw when introducing
#    our datasets, and extend it with .categories(), .words(), etc. meth-
#    ods like those in NLTK's CorpusReaders so that we can use them with-
#    in the same training and testing workflow:
from lib.TwentyNewsgroupsCorpusWrapper import TwentyNewsgroupsCorpusWrapper as twenty_newsgroups

datasets = [brown, reuters, twenty_newsgroups()]


#    We have also implemented a wrapper class 'Classifier' that gives us
#    easy and consistent access to scikit-learn's classification algori-
#    thms:
from lib.Classifier import Classifier

#    Although NLTK's datasets usually come with pre-defined train and test
#    splits of the data, in our experiments we will ignore that distinct-
#    ion and we will be performing cross-validation. When cross-validating,
#    the ratio between training and testing data is observed (for instance,
#    8 training instances for every 2 test instances) but combining diffe-
#    rent parts of the corpus: in the 1st cross-validation fold, the first
#    20% of the dataset is used as training and the remaining 80% for tes-
#    ting; in the 2nd cross-validation fold, testing is performed on the
#    21-40% of the dataset, and training on the remaining 1-20% + 41-100%,
#    and so on. Cross-validation is preferable as an evaluation methodolo-
#    gy because it is far more robust. Results that generalize well to all
#    subsets of our dataset will probably perform well on new data.
#
#    For convenience, we have also implemented a cross-validation wrapper
#    to take care of the experimental design for us:
from lib.CrossValidator import CrossValidator

lr = Classifier(
    classifier='lr',
)

nb = Classifier(
    classifier='mnb',
)

clfs = [lr, nb]


#    Experimental workflow:
for dataset in datasets:
    for clf in clfs:
        c = CrossValidator(clf, dataset, train_r=0.9)
        c.run()


500it [00:04, 119.58it/s]

<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> lr None 0 449 51 0.47 4.26
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> lr None 1 449 51 0.51 4.51
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> lr None 2 449 51 0.41 4.85
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> lr None 3 449 51 0.67 4.4
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> lr None 4 449 51 0.57 3.82
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> lr None 5 449 51 0.51 6.11
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> lr None 6 444 50 0.52 5.49
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> lr None 7 444 50 0.4 5.52
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> lr None 8 446 48 0.52 6.0
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'>


500it [00:03, 144.68it/s]

 lr None 9 355 33 0.64 2.24
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> lr None run 4383 487 0.522 4.72
--------------
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb None 0 449 51 0.24 1.09
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb None 1 449 51 0.2 0.96
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb None 2 449 51 0.24 1.01
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb None 3 449 51 0.25 0.99
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb None 4 449 51 0.22 1.03
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb None 5 449 51 0.16 0.98
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb None 6 444 50 0.22 9.96
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb None 7 444 50 0.26 1.11
<CategorizedTaggedCorpusRea


10788it [00:07, 1487.84it/s]

...

 lr None 9 9445 1059 0.91 11.77
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> lr None run 96256 10730 0.903 14.272
--------------
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb None 0 9656 1132 0.59 2.01
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb None 1 9656 1069 0.63 1.78
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb None 2 9656 1069 0.63 1.52
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb None 3 9656 1069 0.62 1.69
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb None 4 9656 1069 0.63 1.6
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb None 5 9656 1069 0.63 1.44
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb None 6 9631 1064 0.63 1.45
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_dat


19946it [00:10, 1863.77it/s]

 mnb None 9 9445 1059 0.64 1.36
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb None run 96256 10730 0.625 1.569
--------------
<lib.TwentyNewsgroupsCorpusWrapper.TwentyNewsgroupsCorpusWrapper instance at 0x15c4bfe60> lr None 0 17951 1995 0.85 40.8
<lib.TwentyNewsgroupsCorpusWrapper.TwentyNewsgroupsCorpusWrapper instance at 0x15c4bfe60> lr None 1 17951 1995 0.83 37.58
<lib.TwentyNewsgroupsCorpusWrapper.TwentyNewsgroupsCorpusWrapper instance at 0x15c4bfe60> lr None 2 17951 1995 0.84 36.82
<lib.TwentyNewsgroupsCorpusWrapper.TwentyNewsgroupsCorpusWrapper instance at 0x15c4bfe60> lr None 3 17951 1995 0.84 38.34
<lib.TwentyNewsgroupsCorpusWrapper.TwentyNewsgroupsCorpusWrapper instance at 0x15c4bfe60> lr None 4 17951 1995 0.84 37.32
<lib.TwentyNewsgroupsCorpusWrapper.TwentyNewsgroupsCorpusWrapper instance at 0x15c4bfe60> lr None 5 17951 1995 0.84 41.04
<lib.TwentyNewsgroupsCorpusWrapper.TwentyNewsgroupsCorpusWrapper instance at 0x15c4bfe60> lr None 6 17951


19946it [00:08, 2323.14it/s]

 lr None 9 17955 1991 0.84 35.52
<lib.TwentyNewsgroupsCorpusWrapper.TwentyNewsgroupsCorpusWrapper instance at 0x15c4bfe60> lr None run 179514 19946 0.841 38.287
--------------
<lib.TwentyNewsgroupsCorpusWrapper.TwentyNewsgroupsCorpusWrapper instance at 0x15c4bfe60> mnb None 0 17951 1995 0.81 5.04
<lib.TwentyNewsgroupsCorpusWrapper.TwentyNewsgroupsCorpusWrapper instance at 0x15c4bfe60> mnb None 1 17951 1995 0.82 5.2
<lib.TwentyNewsgroupsCorpusWrapper.TwentyNewsgroupsCorpusWrapper instance at 0x15c4bfe60> mnb None 2 17951 1995 0.81 5.0
<lib.TwentyNewsgroupsCorpusWrapper.TwentyNewsgroupsCorpusWrapper instance at 0x15c4bfe60> mnb None 3 17951 1995 0.81 5.11
<lib.TwentyNewsgroupsCorpusWrapper.TwentyNewsgroupsCorpusWrapper instance at 0x15c4bfe60> mnb None 4 17951 1995 0.81 9.27
<lib.TwentyNewsgroupsCorpusWrapper.TwentyNewsgroupsCorpusWrapper instance at 0x15c4bfe60> mnb None 5 17951 1995 0.82 4.86
<lib.TwentyNewsgroupsCorpusWrapper.TwentyNewsgroupsCorpusWrapper instance at 0x15c4bfe60> mnb 


These results are preliminary but very exciting nevertheless. Just to give you a frame of reference, below is a summary of some state-of-the-art results for classification on these corpora (some of these benchmarks are a bit old but I have not been able to find any more recents papers reporting better scores -which definitely does not mean they do not exist):

* [This 2015 paper](https://www.researchgate.net/profile/Giacomo_Domeniconi/publication/281743455_A_Study_on_Term_Weighting_for_Text_Categorization_A_Novel_Supervised_Variant_of_tfidf/links/55f6ad7708aeafc8abf4f470.pdf) reports a maximum accuracy of nearly 94% and average accuracies of 92% on the Reuters corpus using __SVMs, and a number of variations of TFIDF as the feature weighting function, and feature selection__. The same configuration yielded scores in the mid-70% range on the 20 Newsgroups dataset. In our cases, __a barely normalized BoW-based vector space model combined with a Logistic Regression classifier__ allowed us to come very close of their top performance for Reuters (within 3-4%) and to outperform by ~10% their results on the 20 Newsgroups dataset. This is the kind of thing why we said earlier that Logistic Regression is usually the way to go ;)

* On the Brown corpus, one of the top accuracies reported is [a 2010 52%-result](http://www.aclweb.org/anthology/P10-1077) (over all categories, so quite low but definitely better than random -because you have many more categories than just two, which would amount to a 50% random baseline assuming a uniform distribution-). Our out-of-the box, nearly unmodified classifier matched that performance effortlessly, despite the fact that the authors of that paper were using more advanced features. Again, Logistic Regression can be seen to account for the difference (they used Linear Discriminant Analyis, a precursor to Logistic Regression).

* Again on the 20 Newsgroups classifiers, the [Stanford team](http://nlp.stanford.edu) (in many ways, the founders of Natural Language Processing and modern Computational Linguistics) [report a nearly 82% accuracy](http://nlp.stanford.edu/wiki/Software/Classifier/20_Newsgroups) using their classifier. They report some previous results performing up to the ~82% mark, but they also report as "suspect" an earlier result achieving nearly 85% accuracy. Our own results suggest this last result actually seemed on the right track :) (for obvious reasons, Stanford people do not like other people getting better accuracies ;)


##### A note on evaluation metrics
In these experiments, evaluation is being done in terms of [__Accuracy__](https://en.wikipedia.org/wiki/Accuracy_and_precision). Other widely-user important metrics are [__Precision__ and __Recall__](https://en.wikipedia.org/wiki/Precision_and_recall) you should be familiar with.

In NLP, a simplified definition of __accuracy__ is used, not its stronger, technical one. Technically speaking, accuracy refers to how consistent a system's measurements are over time. A system __could be accurate while at the same time delivering really poor performance__ (as long as it delivers consistently poor performance :)

In NLP, results consistency over time is generally measured as __macro-averaged precision__ over several experimental runs, for instance, and __accuracy__ is used as a synonym of precision when precision and recall converge. That is, all three terms can generally be used as equivalent when our task is such that every _X_ has a corresponding _Y_ and the system can either assign the right _Y_ or not, but always assigns one. I will make it clearer in a minute, stay with me ;)

Let's imagine you are pub-questing around Lviv (it could happen ;) Let's imagine the crazy organizers ask you to go to the Opera House and convince 5 Ukrainians of singing the first act of [Rigoletto](https://en.wikipedia.org/wiki/Rigoletto) with you.

Let's assume you get 3 people to do it (by bribing them with ice-cream, for instance ;) but let's also assume there are 3 other people who are not really Ukrainian but have heard you saying you will buy them ice-cream and that they really, really want the ice-cream. Two of them totally look like tourists so you can easily tell them to go away but the third one does look like he might be Ukrainian and you absolutely want to win the pub quest so you decide to risk it.

In this case, your Precision is 2/3 and your Recall is 2/5. The number of actual Ukrainians who joined you (2) is called __true positives__, the number of false non-Ukrainians who joined (1) you is called __false positives__, the people you are missing to achieve the goal (3) are called __false negatives__, and the number of people you managed not to get fooled into buying an ice-cream (2) to is called _true negatives_.

On the other hand, when you are a teacher you get to tell your students to either sing Rigoletto or go home with a note for their parents. Since they will all have to make a decision, __there can be no false negatives__ (i.e., any student who is not classified into singing or not singing); since you cannot force other teacher's students to sing Rigoletto, __there cannot be any false positives__, either; and since the choice is an a/b choice (a dichotomy), the boolean values __true__ and __false__ already account for all the variation, which is why __only Precision applies__ and, as we said, it is used as a synonym of __Accuracy__ in this particular case.

Usually, Precision and Recall are reported using a combined score, their [__harmonic mean or F-1 score__](https://en.wikipedia.org/wiki/F1_score).


## Experiments with a custom __Feature Extractor__

__NOTE__: don't try this at home! :) There seems to be some problem with the implementation of the classifier in the _wlp_ repository and I have not been able to figure out what it is yet. As a result, the Naïve Bayes results are incorrectly low. However, that does not seem to be affecting the feature extraction part because, as far as I am concerned, it is behaving in the usual way.

In [9]:
import nltk
from nltk.corpus import reuters, brown

NGRAMS = [
    (1, False),
    (2, False),
    (3, True)
]

#    We will re-use the SimpleCorpusReader class we saw when introducing
#    our datasets, and extend it with .categories(), .words(), etc. meth-
#    ods like those in NLTK's CorpusReaders so that we can use them with-
#    in the same training and testing workflow:
from lib.TwentyNewsgroupsCorpusWrapper import TwentyNewsgroupsCorpusWrapper as twenty_newsgroups

datasets = [brown, reuters, twenty_newsgroups()]
datasets = [brown, reuters]


#    We have also implemented a wrapper class 'Classifier' that gives us
#    easy and consistent access to scikit-learn's classification algori-
#    thms:
from lib.Classifier import Classifier

#    Although NLTK's datasets usually come with pre-defined train and test
#    splits of the data, in our experiments we will ignore that distinct-
#    ion and we will be performing cross-validation. When cross-validating,
#    the ratio between training and testing data is observed (for instance,
#    8 training instances for every 2 test instances) but combining diffe-
#    rent parts of the corpus: in the 1st cross-validation fold, the first
#    20% of the dataset is used as training and the remaining 80% for tes-
#    ting; in the 2nd cross-validation fold, testing is performed on the
#    21-40% of the dataset, and training on the remaining 1-20% + 41-100%,
#    and so on. Cross-validation is preferable as an evaluation methodolo-
#    gy because it is far more robust. Results that generalize well to all
#    subsets of our dataset will probably perform well on new data.
#
#    For convenience, we have also implemented a cross-validation wrapper
#    to take care of the experimental design for us:
from lib.CrossValidator import CrossValidator

from lib.FeatureExtractor import FeatureExtractor

lr = Classifier(
    classifier='lr'
)

nb = Classifier(
    classifier='mnb'
)

clfs = [nb]

xtor_none = None

xtor_off = FeatureExtractor(
    'off'
)

xtor_rm = FeatureExtractor(
    'rm',
    rm_numbers=True,
    rm_punct=True,
    rm_stopwords=True,
)

xtor_all = FeatureExtractor(
    'battery',
    rm_numbers=True,
    rm_punct=True,
    rm_stopwords=True,
    ngrams=[(1, False), (2, False)],
    lemmatize=True
)

xtors = [xtor_off, xtor_rm, xtor_all]

#    Experimental workflow:
for dataset in datasets:
    for clf in clfs:
        for xtor in xtors:
            c = CrossValidator(clf, dataset, train_r=0.9, extractor=xtor)
#                 c = CrossValidator(clf, dataset, train_r=0.9)
            c.run()

500it [00:07, 68.36it/s]

<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb off 0 449 51 0.2 2.41
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb off 1 449 51 0.25 2.29
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb off 2 449 51 0.2 2.28
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb off 3 449 51 0.22 2.27
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb off 4 449 51 0.2 2.61
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb off 5 449 51 0.27 2.35
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb off 6 444 50 0.26 2.29
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb off 7 444 50 0.2 2.51
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb off 8 446 48 0.23 2.38
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'>


500it [00:05, 87.30it/s]

 mnb off 9 355 33 0.39 1.96
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb off run 4383 487 0.242 2.335
--------------
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb rm 0 449 51 0.27 1.17
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb rm 1 449 51 0.24 1.24
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb rm 2 449 51 0.27 1.31
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb rm 3 449 51 0.27 1.14
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb rm 4 449 51 0.25 1.24
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb rm 5 449 51 0.29 1.19
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb rm 6 444 50 0.3 1.17
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb rm 7 444 50 0.3 1.11
<CategorizedTaggedCorpusReader in u'/Users/


500it [01:54,  4.92it/s]

...

 mnb battery 9 355 33 0.36 5.73
<CategorizedTaggedCorpusReader in u'/Users/jordi/nltk_data/corpora/brown'> mnb battery run 4383 487 0.266 7.384
--------------
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb off 0 9656 1132 0.63 3.93
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb off 1 9656 1069 0.66 3.3
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb off 2 9656 1069 0.67 3.36
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb off 3 9656 1069 0.67 3.95
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb off 4 9656 1069 0.66 3.77
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb off 5 9656 1069 0.67 3.58
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb off 6 9631 1064 0.66 3.53
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/re


10788it [00:09, 1100.71it/s]

 mnb off 9 9445 1059 0.68 3.44
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb off run 96256 10730 0.663 3.595
--------------
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb rm 0 9656 1132 0.69 1.94
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb rm 1 9656 1069 0.73 2.14
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb rm 2 9656 1069 0.73 7.4
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb rm 3 9656 1069 0.72 2.48
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb rm 4 9656 1069 0.73 1.97
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb rm 5 9656 1069 0.74 2.11
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb rm 6 9631 1064 0.72 2.49
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuter


10788it [03:16, 54.90it/s]

 mnb rm 9 9445 1059 0.75 2.06
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb rm run 96256 10730 0.726 2.681
--------------
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb battery 0 9656 1132 0.62 11.35
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb battery 1 9656 1069 0.66 11.33
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb battery 2 9656 1069 0.65 11.39
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb battery 3 9656 1069 0.64 11.2
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb battery 4 9656 1069 0.64 11.65
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb battery 5 9656 1069 0.65 11.84
<CategorizedPlaintextCorpusReader in u'/Users/jordi/nltk_data/corpora/reuters'> mnb battery 6 9631 1064 0.65 11.88
<CategorizedPlaintextCorpusReader in

As you can see, I was not lying :) We got a macro-averaged improvement of ~6% over the Brown and the Reuters corpora without doing anything fancy, just some data clean-up.


## Sentiment analysis

For the purposes of NLP, [__sentiment analysis__](https://en.wikipedia.org/wiki/Sentiment_analysis) is just a binary or ternary classification problem: binary when classifying into __positive__ or __negative__ sentiment, ternary for tasks also involving a third class for __neutral__ sentiment (neither positive or negative). The terms __positive__ or __negative__ here refer to the subjective notion of goodness: _Daenerys Targaryen = good_, _Cersei Lannister = bad_.

In terms of algorithms and workflow, the task remains essentially the same as standard document __classification__, which means that most of the time an actual implementation will consist in training a classifier over as much data as we can lay our hands on :), then run new data through the model.

For sentiment analysis (and classification more generally), data size is very likely to be the key performance factor. Feature engineering can help but probably less than just throwing more data at the system, at least in terms of getting quick gains. This is partly because sentiment analysis is a relatively vague task with a lot of long-tail variation but a very well-defined subset of frequent terms used to express positive or negative sentiment.

That is, every language has a limited set of swearwords that all speakers learn pretty fast (twice as fast if you are a foreigner :) Given enough data, the system can easily learn that list of words from the training examples. Once you know the term _freaking disgrace_, there is little ambiguity left that could mislead the system :) A similar pattern applies to positive sentiment -the information we want to capture in both cases is in the words themselves (and can therefore be modelled using a BoW-model) rather than in higher-order information like combinations of those words.

Let's see some sentiment analysis in action:

In [10]:
import nltk

from lib.Classifier import Classifier
from lib.CrossValidator import CrossValidator
from lib.FeatureExtractor import FeatureExtractor

from lib.Sentiment import Sentiment
from lib.SentimentCorpusReaderWrapper import SentimentCorpusReaderWrapper as sentiment_corpus

NGRAMS = [
    (1, False),
    (2, False),
    (3, True),
]

datasets = [sentiment_corpus()]

sentiment = Sentiment('sentiwordnet')

lr = Classifier(
    classifier='lr',
)

nb = Classifier(
    classifier='mnb',
)

clfs = [lr]

xtor_none = None

xtor_off = FeatureExtractor(
    'off'
)

xtor_clean = FeatureExtractor(
    'clean',
    rm_numbers=True,
    rm_punct=True,
)

xtor_grams = FeatureExtractor(
    'ngrams',
    rm_numbers=True,
    rm_punct=True,
    ngrams=NGRAMS,
)

xtor_sent = FeatureExtractor(
    'sentiment',
    rm_numbers=True,
    rm_punct=True,
    sentiment=sentiment
)


xtors = [xtor_off, xtor_clean, xtor_sent, xtor_grams]

#    Experimental workflow:
for clf in clfs:
    for dataset in datasets:
        for xtor in xtors:
            c = CrossValidator(clf, dataset, train_r=0.8, extractor=xtor)
            c.run()

50000it [00:03, 14392.40it/s]

<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr off 0 40000 10000 0.77 2.01
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr off 1 40000 10000 0.76 5.02
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr off 2 40000 10000 0.76 2.09
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr off 3 40000 10000 0.77 2.27
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830>


50000it [00:06, 7375.93it/s]

 lr off 4 40000 10000 0.77 4.76
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr off run 200000 50000 0.766 3.23
--------------
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr clean 0 40000 10000 0.76 2.35
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr clean 1 40000 10000 0.76 2.38
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr clean 2 40000 10000 0.76 5.17
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr clean 3 40000 10000 0.76 1.86
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830>


50000it [00:06, 7459.06it/s] 

 lr clean 4 40000 10000 0.76 1.85
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr clean run 200000 50000 0.76 2.722
--------------
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr sentiment 0 40000 10000 0.76 5.33
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr sentiment 1 40000 10000 0.76 9.07
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr sentiment 2 40000 10000 0.76 5.5
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr sentiment 3 40000 10000 0.77 5.68
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830>


50000it [00:08, 5967.61it/s]

 lr sentiment 4 40000 10000 0.76 8.92
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr sentiment run 200000 50000 0.762 6.9
--------------
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr ngrams 0 40000 10000 0.77 12.1
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr ngrams 1 40000 10000 0.77 18.51
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr ngrams 2 40000 10000 0.77 12.49
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr ngrams 3 40000 10000 0.77 12.27
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr ngrams 4 40000 10000 0.77 12.1
<lib.SentimentCorpusReaderWrapper.SentimentCorpusReaderWrapper instance at 0x19d674830> lr ngrams run 200000 50000 0.77 13.494
--------------


##### A very important lesson
So, two things are important here: first, the results this time are awful -we are very likely __not__ beating anyone's state-of-the-art performance this time XD If you remember the structure of our sentiment dataset (which contained only __positive__ and __negative__ labels), we are performing binary classification so a random baseline would give us 0.5 accuracy. We are close to 0.8 but not even there yet, which means that our model is definitely better than random but not extremely better.

The other important thing to note here is that features harmed performance more than anything else -it definitely seems that, in the case of sentiment analysis, whatever the system needs to learn that is going to help it determine whether something is positive or negative, is already there, regardless of our current features and regardless of our polarity dictionaries. This implies that, as long as we have data available, we do not really need to do anything with it.

The key lessons here are:
1. Algorithms will tend to see correlations in the data more easily than we can, which means that, theoretically, they can determine whether two variables are significantly associated more reliably than we can. As long as the __data is clean__, they will find whatever we want them to find. Yet again, another reason to take very seriously our earlier point about knowing the data.
2. Additional data will maximize Recall and, given enough data, we will not be able to teach our system with a dictionary anything it has not learned from the corpus already. In the battle for classification, corpus always beats dictionary.



### Dictionaries

Traditionally, __polarity dictionaries__ have been used to assign a sentiment to an input text based on the sentiment of the words in that text. In our _wlp_ repository I have included a class called __Sentiment__ illustrating the use of a good range of existing dictionary[1]-based sentiment analysis libraries for Python: __[Afinn](https://pypi.python.org/pypi/afinn), [SenticNet](https://pypi.python.org/pypi/senticnet/0.5.0), [SentiWordNet__](http://sentiwordnet.isti.cnr.it), and __Vader__ (the last one comes with NLTK and I have included a copy of SentiWordNet in the repository, but the other two should be installed. There is a good chance I forgot to ask the organizers to do it ;) so feel free to run _for lib in sentic afinn; do pip install $lib; done_ from the command line whenever you have a second if you want to try the code).

The fact that data is usually enough to get a prototype running does not mean that there are not some additional types of information that would also be necessary to correctly understand sentiment. Complex phenomena exist that it would be great to be able to model, but in general there are practical limitations as to how much complexity you want to build into the system: fine-grained sentiment detection would require a deep understanding of syntactic structure on the part of the system, which is a non-trivial task (more on that in a bit). It is also strongly domain-dependent, as well, which means that you would normally need to focus on a particular domain to be able to improve your system significantly.

[1] In this context, dictionaries are also called _lexicons_ in the field of NLP.

In [11]:
from lib.Sentiment import (
    Sentiment
)

terms = 'good old bad great wonderful terrible unknown'

for resource in [Sentiment('afinn'), Sentiment('sentiwordnet'), Sentiment('vader'), Sentiment('sentic')]:
    for term in terms.split():
        print resource, term, resource.word_sentiment(term)



sample_amazon_review = """So bad it was good. The movie is slightly brain-dead and full of barely hidden tropes.
I started to enjoy it when I stopped taking any part of it seriously and just watched the director blow things up.
Runs long. Its like 2 mini-movies, a traditional Transformers movie first, then a Transformers teams with Godzilla
smashup in China. It got that awesomely weird. If you like funny and weird with explosions you will love this.

For those that like a little social commentary- it is a reflection of our currently tortured relationship with the
hero-warrior archetype. We want to watch the hero warrior even though it feels thoroughly discredited. So it was
reincarnated as a robot, an ALIEN robot. HELLO, OPTIMUS PRIME! And this particular movie goes further, surrounding
the unreconstructed hero with all sorts of examples of hero gone bad from the cruel self-centered CIA director to
the clueless, greedy best friend . The human lead, Mark Wahlburg, even has a line: "Humans make mistakes, that's
what we do." Sadly there are days when I can relate all too clearly."""


for resource in [Sentiment('afinn'), Sentiment('sentiwordnet'), Sentiment('vader')]:
    print resource, '#review', resource.text_sentiment(sample_amazon_review)
    print resource, '#terms', resource.text_sentiment(terms)
    

afinn good 0.858571428571
afinn old 0.43
afinn bad 0.00142857142857
afinn great 0.858571428571
afinn wonderful 1.0
afinn terrible 0.00142857142857
afinn unknown 0.43
sentiwordnet good 0.51
sentiwordnet old 0.01
sentiwordnet bad 0
sentiwordnet great 0.01
sentiwordnet wonderful 0.01
sentiwordnet terrible 0
sentiwordnet unknown 0
vader good 1.0
vader old 0.01
vader bad 0
vader great 1.0
vader wonderful 1.0
vader terrible 0
vader unknown 0.01
sentic good 0.945
sentic old 0.49
sentic bad 0.32
sentic great 0.93
sentic wonderful 0.905
sentic terrible 0.05
sentic unknown 0.455
afinn #review 0.43
afinn #terms 1.0
sentiwordnet #review 0.0395027624309
sentiwordnet #terms 0.0771428571429
vader #review 0.04
vader #terms 0.22


### Challenges of sentiment analysis

As we said, fine-grained, high-quality sentiment analysis is a tricky business and it would require more than just a classifier trained on BoW model. Apart from the often-cited issues stemming from figurative language use such as sarcasm, irony, or metaphors, and apart from the standard problems of all NLP-related tasks (co-references, ambiguity), some of the basic aspects of language also pose challenges of their own. Just to name a few:

1. __Negation__. Weird as it may seem, this still poses a problem for many NLP systems. Given that it is a logical rather than a statistical phenomenon, it is difficult to teach a system to interpret natural language negations in a quantitative manner. Usually, most systems end up implementing some rule-based approach for dealing with negation, such as switching the polarity of some number of words following a negation term. The challenge, as usual in NLP, is linguistic variation: there are just too many ways of negating something.

2. __Modulated polarity__. If you remember, during the introduction we mention the phenomenon of __modulation__, the fact that some words' meaning is actually updated in context and does not fully exist independently (except in a very abstract representation). This is also true in sentiment analysis, where the polarity of some words depends on the context in which they appear: _high resolution_ is great, whereas _high price_ is not great at all. There is definitely sentiment involved in these expressions, although it is neither explicit or trivial, that is, there is no unambiguous marker that the system can learn from the data (the _freaking disgrace_ example from above). The sentiment is built compositionally by our minds while interpreting the semantic content of the terms involved: _price_ is generally an objective reality (neutral), but the commonsense-knowledge assumption is that humans do not want to pay high prices because they do not want to spend money. This is the kind of knowledge most systems still do not have, although a very successful paradigm has been recently introduced that is already able to handle some of these phenomena: __aspect-based sentiment analysis__. Each concept is decomposed into its salient aspects (price and resolution in the case of a screen, for instance), and the model training incorporates information as to which words refer to each aspect independently and, therefore, which ones are positive for one of the aspects and may not be for the other.

3. __Domain-specificity__. A lot of sentiment-related information requires domain-specific data (or dictionaries). Some things could be seen as negative in a domain but not in another. A sentence like _The company shipped 10,000 jobs overseas_ would be probably negative in a text about politics. In a financial report, however, it could be positive in a news item about cutting costs, or negative in a news item about national employment, or rather: negative for the company forced to do the cutting, neutral for the network that is reporting the news, and positive as a measure that may still help save the struggling company. Sentiment is a by definition a subjective category and an oversimplification of what goes on in people's minds. Currently, most sentiment analysis technology can just be seen as __swearword detectors__ :) They can detect obvious mentions of explicit sentiment, but they have no clue as to the potential semantic interactions modulating the final sentiment of a given text. For that, an in-depth knowledge (=data) of the domain is required and, for very specific tasks or narrow domains, that information is usually not there. In all those cases, lexicon-driven approaches will be a good way to fill the gaps. When enough data is available, though, no amount of dictionaries should get in the way of training a Logistic Regression classifier :)